In [ ]:
import numpy as np 
from matplotlib import pyplot
%matplotlib inline


In [ ]:
from os.path import expanduser
home = expanduser('~')
gc_dir = home + "/Dropbox/GalacticCenter/"

<h2> Extract and plot points from lightcurve 

In [ ]:
# set the energy limits 
E = 0 

In [ ]:
# extract the lightcurve data points

lc_data = np.genfromtxt(gc_dir+"/results/lightcurve_SgrA_daily_points_v254.txt") 
#lc_data = np.genfromtxt(gc_dir+"/log/lightCurves/SgrA_lightCurve_both_4tels_all_E"+str(E)+"_annual_points.txt")
#lc_data = np.genfromtxt(gc_dir+"/log/lightCurves/SgrA_lightCurve_both_E"+str(E)+"_annual_points.txt")


MJD_array = np.array(lc_data[:,0], dtype=np.float64)
flux_obs_array = np.array(list(filter((0.0).__ne__, lc_data[:,1])), dtype=np.float64)
flux_err_array = np.array(list(filter((0.0).__ne__, lc_data[:,2])), dtype=np.float64)
livetime_array = np.array(list(filter((0.0).__ne__, lc_data[:,3])), dtype=np.float64)

print(flux_obs_array)
#log = list(map(lambda l: np.log10(l), _array))


In [ ]:
# find the total livetime and mean flux 
total_livetime = np.sum(livetime_array)
fluxtime_array = flux_obs_array * livetime_array
mean_flux = np.sum(fluxtime_array) / total_livetime
print(mean_flux)


In [ ]:
#flux_obs_array[ flux_obs_array==0 ] = np.nan
#flux_err_array[ flux_err_array==0 ] = np.nan
#fluxtime_array[ fluxtime_array==0 ] = np.nan

# remove empty points, marked as 0's 
#flux_obs_array = list(filter((0.0).__ne__, flux_obs_array))
#flux_err_array = list(filter((0.0).__ne__, flux_err_array))
#livetime_array = list(filter((0.0).__ne__, livetime_array))


In [ ]:
print(flux_obs_array)

In [ ]:
pyplot.errorbar(MJD_array, flux_obs_array, yerr=flux_err_array, fmt='b.') # 

pyplot.title("Sgr A* Integral Flux ("+str(E)+"TeV < E < 100TeV)")
pyplot.xlabel(r"$time\left(MJD\right)$")
pyplot.ylabel(r"flux $\frac{gamma}{m^{2}s}$") # gamma / m^2*s*TeV
#pyplot.xlim(1.5e0, 5.e1)
pyplot.ylim(ymin=0.)
pyplot.axhline(mean_flux)

#! MAY NEED TO REPLOT E0?  
#pyplot.savefig(gc_dir+"/plots/lightCurve/SgrA_lightcurve_all_4tels_seasonal.png")
#pyplot.savefig(gc_dir+"/plots/lightCurve/SgrA_lightcurve_all_4tels_E"+str(E)+"_seasonal.png")


<h2> Sliding window

In [ ]:
from math import sqrt, erfc

def S_win(Nw, No, alpha):
    """calculates significance of deviation of flux 
    Nw: excess counts inside window
    No: excess counts outside window
    alpha: ratio of window duration to duration outside of window"""
    
    return (Nw - alpha*No)/sqrt(Nw + alpha**2*No)

def P_pre(S):
    """the chance probability calculated from significance for a window"""
    
    return erfc(S/sqrt(2))

def P_post(P_pre, N):
    """post-trials probability after applying trials factors"""

    return 1 - (1-P_pre)**N


In [ ]:
N = len(lc_data)
n = 2 # window size 

No = np.sum(fluxtime_array)
Sig_windows = []

for i in range(N):
    Nw = np.sum(fluxtime_array[i:i+2])
    alpha = np.sum(livetime_array[i:i+2]) / total_livetime
    Sig_windows.append(S_win(Nw, No, alpha)) 
    
print(Sig_windows)
pyplot.hist(Sig_windows)


In [ ]:
# fit hist to Gaussian 
from scipy.optimize import curve_fit

def gauss_norm(x, x0, sigma, A=1):
    """normalized 1D gaussian function"""
    return A/(np.sqrt(2*np.pi))*np.exp(-((x-x0)/sigma)**2/2)



In [ ]:
p0 = (1, 0, 1)

p_opt, p_cov = curve_fit(gauss_norm(x, y, p0))


In [ ]:
print(gauss_norm(2, 0, 1))